In [1]:
ENV["PYTHONPATH"] = "/home/gridsan/aligho/.local/lib/python3.8/site-packages/PyNormaliz-2.15-py3.8-linux-x86_64.egg";
D = 2;
using HDF5, PyPlot
using DelimitedFiles, PyPlot, LinearAlgebra, JLD2, PyCall, StaticArrays, Statistics
using Crystalline, Brillouin, MPBUtils, SymmetryBases
using Crystalline: TEST_αβγs, TEST_αβγ, dot, norm
using Crystalline: normscale, matching_littlegroups
using Crystalline: AbstractFourierLattice
topology_paper_dir = "../TopologyPaper/"
include(topology_paper_dir * "get-freqs-symeigs.jl")
include(topology_paper_dir * "symeigs-from-io.jl");
include("../Topology_ML/Phc_ML/fft.jl")
include("get_uc_coefs.jl");

In [2]:
data_dir = "../TopologyPaper/symeigs/"
filename = "sg2-data-g0.h5";

In [3]:
loaded_data_tm = Dict{String, Any}[]
for epsid in 1:5
    dir = data_dir * "./output/sg2/eps$epsid/tm/";
    filename_jld = "sg2-epsid$(epsid)-res64-tm.jld2"
    loaded_data_per_epsid = load(dir * filename_jld)
    push!(loaded_data_tm, loaded_data_per_epsid)
end

In [4]:
loaded_data_te = Dict{String, Any}[]
for epsid in 1:5
    dir = data_dir * "./output/sg2/eps$epsid/te/";
    filename_jld = "sg2-epsid$(epsid)-res64-te.jld2"
    loaded_data_per_epsid = load(dir * filename_jld)
    push!(loaded_data_te, loaded_data_per_epsid)
end

In [7]:
summariesv_tm = vcat([x["summariesv"] for x in loaded_data_tm]...);

first_band_summaries_tm = [summaries[1] for summaries in summariesv_tm];
second_band_summaries_tm = [summaries[2] for summaries in summariesv_tm];
third_band_summaries_tm = [summaries[3] for summaries in summariesv_tm];
first_band_topologies_tm = [summary.topology for summary in first_band_summaries_tm];
band_topology_count_tm = [count(x-> x.topology == NONTRIVIAL, [summaries[band_idx] for summaries in summariesv_tm]) for band_idx in 1:3] ./ 500

3-element Vector{Float64}:
 28.596
 36.536
 46.856

In [12]:
summariesv_te = vcat([x["summariesv"] for x in loaded_data_te]...);

first_band_summaries_te = [summaries[1] for summaries in summariesv_te];
second_band_summaries_te = [summaries[2] for summaries in summariesv_te];
third_band_summaries_te = [summaries[3] for summaries in summariesv_te];
first_band_topologies_te = [summary.topology for summary in first_band_summaries_te];
band_topology_count_te = 100 * [count(x-> x.topology == NONTRIVIAL, [summaries[band_idx] for summaries in summariesv_te]) for band_idx in 1:3] ./ length(summariesv_te)

3-element Vector{Float64}:
 41.286
 44.35
 48.216

In [19]:
#band_summaries = first_band_summaries_tm
band_summaries = first_band_summaries_te

#band_summaries = second_band_summaries
#band_summaries = third_band_summaries

y_plus_idxs = findall(x->x[1] == 1 && x[end] == 1, [x.n for x in band_summaries]);
y_minus_idxs = findall(x->x[2] == 1 && x[end] == 1, [x.n for x in band_summaries]);

b_plus_idxs = findall(x->x[3] == 1 && x[end] == 1, [x.n for x in band_summaries]);
b_minus_idxs = findall(x->x[4] == 1 && x[end] == 1, [x.n for x in band_summaries]);

a_plus_idxs = findall(x->x[5] == 1 && x[end] == 1, [x.n for x in band_summaries]);
a_minus_idxs = findall(x->x[6] == 1 && x[end] == 1, [x.n for x in band_summaries]);

gamma_plus_idxs = findall(x->x[7] == 1 && x[end] == 1, [x.n for x in band_summaries]);
gamma_minus_idxs = findall(x->x[8] == 1 && x[end] == 1 , [x.n for x in band_summaries]);

In [15]:
f = h5open(topology_paper_dir  * "sg2-data-g0.h5", "r");

In [16]:
epsilon_Gs = vcat([[f["sg2/$(i)/epsilon_Gs-epsid=$(epsid)"][] for i in 1:10000] for epsid in 1:5]...);

In [17]:
epsilon_Gs_signs = [real.(x ./abs.(x)) for x in epsilon_Gs];

In [21]:
round.(100 * [count(x -> x[i] < 0,  epsilon_Gs_signs[b_plus_idxs]) + count(x -> x[i] >  0,  
            epsilon_Gs_signs[b_minus_idxs]) for i in 2:10] / (length(b_plus_idxs)+length(b_minus_idxs)), digits=0)

9-element Vector{Float64}:
 50.0
 76.0
 50.0
 50.0
 51.0
 51.0
 51.0
 52.0
 50.0

In [24]:
round.(100 * [count(x -> x[i] > 0,  epsilon_Gs_signs[y_plus_idxs]) + count(x -> x[i] <  0,  
            epsilon_Gs_signs[y_minus_idxs]) for i in 2:10] / (length(y_plus_idxs)+length(y_minus_idxs)), digits=0)

9-element Vector{Float64}:
 21.0
 50.0
 50.0
 50.0
 50.0
 50.0
 51.0
 50.0
 49.0

# Delineating by contrast 

In [45]:
epsid = 5
#band_summaries = first_band_summaries[(1+(epsid-1)*10000):((epsid*10000))]
band_summaries = first_band_summaries_te[(1+(epsid-1)*10000):((epsid*10000))]


y_plus_idxs = findall(x->x[1] == 1 && x[end] == 1, [x.n for x in band_summaries]);
y_minus_idxs = findall(x->x[2] == 1 && x[end] == 1, [x.n for x in band_summaries]);

b_plus_idxs = findall(x->x[3] == 1 && x[end] == 1, [x.n for x in band_summaries]);
b_minus_idxs = findall(x->x[4] == 1 && x[end] == 1, [x.n for x in band_summaries]);

a_plus_idxs = findall(x->x[5] == 1 && x[end] == 1, [x.n for x in band_summaries]);
a_minus_idxs = findall(x->x[6] == 1 && x[end] == 1, [x.n for x in band_summaries]);

gamma_plus_idxs = findall(x->x[7] == 1 && x[end] == 1, [x.n for x in band_summaries]);
gamma_minus_idxs = findall(x->x[8] == 1 && x[end] == 1 , [x.n for x in band_summaries]);

In [46]:
round.(100 * [count(x -> x[i] <  0,  epsilon_Gs_signs[(1+(epsid-1)*10000):((epsid*10000))][b_plus_idxs]) + count(x -> x[i] >  0,  
            epsilon_Gs_signs[(1+(epsid-1)*10000):((epsid*10000))][b_minus_idxs]) for i in 2:10] / (length(b_plus_idxs)+length(b_minus_idxs)), digits=0)

9-element Vector{Float64}:
 50.0
 70.0
 50.0
 50.0
 51.0
 51.0
 52.0
 52.0
 50.0